In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

# Data Preprocessing

In [2]:
import pandas as pd

import utils

In [3]:
# Read the Redfin data into a DataFrame
file_path_redfin1 = "../data/redfin_data1.csv"
file_path_redfin2 = "../data/redfin_data2.csv"
df_redfin1 = pd.read_csv(file_path_redfin1)
df_redfin2 = pd.read_csv(file_path_redfin2)
df_redfin = pd.concat([df_redfin1, df_redfin2])
df_redfin["period_begin"] = pd.to_datetime(df_redfin["period_begin"])

#make it easier to merge with av_data
df_redfin["date"] = df_redfin["period_begin"] + pd.Timedelta(days=-3)
df_redfin = df_redfin.drop(columns=["period_begin", "period_end"])
df_redfin

,Unnamed: 0,region_name,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,...,months_of_supply,months_of_supply_yoy,median_pending_sqft,median_pending_sqft_yoy,average_sale_to_list_ratio,average_sale_to_list_ratio_yoy,median_sale_ppsf,median_sale_ppsf_yoy,State,date
0,2,Crook County,5.0,-0.375000,8.0,0.333333,1.0,-0.500000,9.0,3.500000,...,12.000000,-60.000000,1618.0,-0.071449,0.937917,0.021480,150.000000,0.218853,OR,2017-01-27
1,10,Chambers County,15.0,0.666667,14.0,0.076923,3.0,-0.250000,11.0,0.833333,...,13.363636,-10.469697,2227.0,0.183316,0.973733,-0.015372,94.778849,-0.035945,TX,2017-01-27
2,32,St. Joseph County,12.0,-0.555556,17.0,0.000000,1.0,-0.500000,13.0,-0.133333,...,20.153846,-3.512821,1500.0,-0.417363,0.933705,0.005458,61.458333,0.156474,MI,2017-02-03
3,42,Crook County,6.0,-0.400000,12.0,-0.142857,1.0,-0.500000,1.0,-0.888889,...,107.000000,90.666667,1984.0,0.634267,1.000000,-0.009948,143.509615,0.506180,OR,2017-02-03
4,69,Coos County,16.0,-0.360000,23.0,-0.080000,5.0,-0.166667,11.0,-0.450000,...,28.818182,13.268182,1760.0,0.085415,0.941134,-0.017685,116.892373,-0.041654,OR,2017-03-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166274,4868262,Troup County,15.0,0.363636,20.0,1.857143,1.0,0.000000,18.0,0.200000,...,14.222222,-3.111111,1648.0,-0.208833,0.988753,-0.010559,146.915071,0.086005,GA,2024-11-15
166275,4868265,Nez Perce County,7.0,0.400000,6.0,0.200000,2.0,-0.333333,8.0,1.000000,...,12.375000,-10.875000,2860.0,0.914324,0.999153,0.041135,189.274448,-0.264456,ID,2024-11-15
166276,4868271,Coles County,17.0,1.833333,11.0,0.222222,4.0,3.000000,9.0,2.000000,...,10.555556,-19.777778,1519.5,-0.038291,0.941275,-0.056837,104.166667,0.351781,IL,2024-11-15
166277,4868277,Marion County,6.0,-0.142857,4.0,-0.500000,4.0,1.000000,12.0,1.400000,...,8.166667,-6.433333,1098.5,-0.147127,0.966881,-0.046709,214.650323,-0.145966,IA,2024-11-15


In [4]:
#Inspect Data
df_redfin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 332558 entries, 0 to 166278
Data columns (total 36 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   Unnamed: 0                                    332558 non-null  int64         
 1   region_name                                   332558 non-null  object        
 2   adjusted_average_new_listings                 332558 non-null  float64       
 3   adjusted_average_new_listings_yoy             332558 non-null  float64       
 4   average_pending_sales_listing_updates         332558 non-null  float64       
 5   average_pending_sales_listing_updates_yoy     332558 non-null  float64       
 6   off_market_in_two_weeks                       332558 non-null  float64       
 7   off_market_in_two_weeks_yoy                   332558 non-null  float64       
 8   adjusted_average_homes_sold                   332558 non-nu

In [5]:
# Read the Zillow data into a DataFrame
file_path_zillow_zori = "../data/County_zori_uc_sfrcondomfr_sm_month.csv"
file_path_zillow_zhvi = "../data/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
df_zillow_zori = pd.read_csv(file_path_zillow_zori)
df_zillow_zhvi = pd.read_csv(file_path_zillow_zhvi)

In [6]:
#Extract required features
df_zillow_zori.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"],
                    inplace=True)
df_zillow_zhvi.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"],
                    inplace=True)

#restructure so that the dates become rows
df_zillow_zori = pd.melt(df_zillow_zori, id_vars=["RegionName", "State", "Metro"],
                         var_name="Date",
                         value_name="ZORI")
df_zillow_zhvi = pd.melt(df_zillow_zhvi, id_vars=["RegionName", "State", "Metro"],
                         var_name="Date",
                         value_name="ZHVI")

#apply time horizon
df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format=True)
df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format=True)
df_zillow_zhvi = df_zillow_zhvi[df_zillow_zhvi["Date"] > "2015-01-31"]

#merge
df_zillow = pd.merge(df_zillow_zhvi, df_zillow_zori[["RegionName", "State", "Metro", "Date", "ZORI"]],
                     on=["RegionName", "State", "Metro", "Date"],
                     how="left")
df_zillow.dropna(inplace=True)

#rename column to match df_redfin
df_zillow.rename(columns={"RegionName": "region_name"}, inplace=True)
df_zillow.rename(columns={"Date": "date"}, inplace=True)

#resample to weekly
df_zillow["date"] += pd.to_timedelta(df_zillow.groupby("date").cumcount(), unit="s")
df_zillow.set_index("date", inplace=True)
df_zillow = df_zillow.groupby("region_name").resample("W-FRI").ffill()
df_zillow = df_zillow.drop(columns=["region_name"]).reset_index()
df_zillow

/var/folders/8b/9mzlkxlx3zx9nmpy8prjpmm80000gn/T/ipykernel_40260/1154634796.py:14: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format = True)
/var/folders/8b/9mzlkxlx3zx9nmpy8prjpmm80000gn/T/ipykernel_40260/1154634796.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format = True)


,region_name,date,State,Metro,ZHVI,ZORI
0,Ada County,2015-03-06,ID,"Boise City, ID",204657.323819,948.248180
1,Ada County,2015-03-13,ID,"Boise City, ID",204657.323819,948.248180
2,Ada County,2015-03-20,ID,"Boise City, ID",204657.323819,948.248180
3,Ada County,2015-03-27,ID,"Boise City, ID",204657.323819,948.248180
4,Ada County,2015-04-03,ID,"Boise City, ID",205924.714086,956.009824
...,...,...,...,...,...,...
250238,Yuma County,2024-10-04,AZ,"Yuma, AZ",277673.650150,1473.600844
250239,Yuma County,2024-10-11,AZ,"Yuma, AZ",277673.650150,1473.600844
250240,Yuma County,2024-10-18,AZ,"Yuma, AZ",277673.650150,1473.600844
250241,Yuma County,2024-10-25,AZ,"Yuma, AZ",277673.650150,1473.600844


In [7]:
#Inspect Data
df_zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250243 entries, 0 to 250242
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   region_name  250243 non-null  object        
 1   date         250243 non-null  datetime64[ns]
 2   State        250198 non-null  object        
 3   Metro        250198 non-null  object        
 4   ZHVI         250198 non-null  float64       
 5   ZORI         250198 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 11.5+ MB


In [8]:
#Read alpha vantage data
file_path_av = "../data/av_data.csv"
df_av = pd.read_csv(file_path_av)
df_av["date"] = pd.to_datetime(df_av["date"])
df_av

,Unnamed: 0,date,SPY_adj_close,SPY_return_1day,SPY_return_1week,SPY_return_1month,SPY_return_3month,EQR_adj_close,EQR_adj_close_1day,EQR_adj_close_1week,...,AVB_adj_close_3month,INVH_adj_close,INVH_adj_close_1day,INVH_adj_close_1week,INVH_adj_close_1month,INVH_adj_close_3month,unemployment,fed_funds,3monhth_treasury_yield,cpi
0,8,2017-02-03,201.410539,-0.001788,0.009462,0.036496,0.045656,45.852975,45.989668,47.030055,...,148.539775,17.272979,17.356707,17.599516,18.353064,18.018154,0.047,0.000098,0.0065,2.42839
1,9,2017-02-10,203.316271,0.005443,0.015464,0.027213,0.038761,47.030055,47.189531,46.764263,...,149.262275,17.599516,17.582771,17.708362,17.900935,17.959545,0.046,0.000104,0.0066,2.43603
2,10,2017-02-17,206.460293,0.005955,0.008592,0.011536,0.025677,46.764263,47.402165,48.040066,...,149.238969,17.708362,17.976290,18.260964,18.034899,17.941583,0.046,0.000102,0.0066,2.43603
3,11,2017-02-24,207.909361,0.001563,0.007096,-0.007860,0.016153,47.675551,48.040066,47.326224,...,149.137974,18.085136,18.260964,18.260964,18.068390,18.193454,0.046,0.000100,0.0066,2.43603
4,12,2017-03-03,209.384761,-0.002978,-0.003062,-0.006931,0.017333,47.326224,47.402165,47.052837,...,148.633001,18.260964,18.353064,17.917681,18.277709,17.933188,0.046,0.000117,0.0065,2.43603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,389,2024-08-16,552.612504,0.009561,0.014108,0.015389,0.083508,70.775391,70.953791,72.975662,...,232.210000,35.000079,35.347302,35.863177,36.478257,33.830000,0.042,0.000999,0.0533,3.14796
382,390,2024-08-23,560.408556,-0.002384,0.002757,0.016526,0.049502,72.975662,72.281883,74.214553,...,229.220000,35.863177,35.545716,36.547702,36.210400,34.150000,0.042,0.000990,0.0533,3.14796
383,391,2024-08-30,561.953809,-0.020579,-0.030638,0.021009,0.063308,74.214553,74.987622,74.432598,...,232.900000,36.547702,36.974290,35.406826,35.260000,33.840000,0.042,0.000979,0.0533,3.14796
384,392,2024-09-06,538.705224,0.011196,0.040066,0.063624,0.120520,73.679352,74.432598,77.386115,...,230.800000,35.664763,35.406826,36.726274,34.540000,33.900000,0.041,0.000968,0.0533,3.15301


In [9]:
#Inspect Data
df_av.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              386 non-null    int64         
 1   date                    386 non-null    datetime64[ns]
 2   SPY_adj_close           386 non-null    float64       
 3   SPY_return_1day         386 non-null    float64       
 4   SPY_return_1week        386 non-null    float64       
 5   SPY_return_1month       386 non-null    float64       
 6   SPY_return_3month       386 non-null    float64       
 7   EQR_adj_close           386 non-null    float64       
 8   EQR_adj_close_1day      386 non-null    float64       
 9   EQR_adj_close_1week     386 non-null    float64       
 10  EQR_adj_close_1month    386 non-null    float64       
 11  EQR_adj_close_3month    386 non-null    float64       
 12  ESS_adj_close           386 non-null    float64   

In [10]:
#merge redfin and zillow data source
df = pd.merge(df_redfin, df_zillow[["region_name", "State", "date", "ZORI", "ZHVI"]],
              on=["region_name", "State", "date"],
              how="left")
df = df.drop(columns=['Unnamed: 0'])

'''df = pd.merge(df, df_av,
                     on = [ "date"],
                     how = "left")'''
df.dropna(inplace=True)
df

,region_name,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,median_new_listing_price,...,median_pending_sqft,median_pending_sqft_yoy,average_sale_to_list_ratio,average_sale_to_list_ratio_yoy,median_sale_ppsf,median_sale_ppsf_yoy,State,date,ZORI,ZHVI
12,Arapahoe County,162.0,-0.129032,160.0,-0.111111,59.0,0.311111,138.0,-0.126582,318950.0,...,1967.0,-0.028162,0.989641,-0.001721,151.855369,0.097923,CO,2016-12-30,1306.117777,325881.246176
16,Stanislaus County,163.0,0.108844,152.0,-0.061728,52.0,-0.187500,112.0,-0.157895,289000.0,...,1574.0,0.019430,0.983747,-0.000931,167.416218,0.110304,CA,2017-03-10,1169.349237,270818.216906
18,Stearns County,73.0,0.123077,42.0,0.000000,11.0,0.100000,31.0,0.033333,168500.0,...,1938.0,-0.021212,0.969423,-0.006978,87.180544,0.111337,MN,2017-03-10,890.363833,186465.879784
25,Washoe County,73.0,-0.401639,119.0,-0.201342,11.0,-0.312500,85.0,-0.174757,313500.0,...,1534.0,0.009875,0.975629,-0.001425,171.276080,0.051403,NV,2016-12-30,1148.605311,305904.466295
26,Chittenden County,32.0,0.142857,26.0,-0.277778,5.0,0.000000,23.0,-0.148148,302450.0,...,1762.5,0.056022,0.986289,0.010741,138.162307,-0.103745,VT,2017-01-06,1714.842956,304940.863948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332534,Cuyahoga County,397.0,-0.076744,405.0,-0.093960,150.0,-0.202128,292.0,-0.165714,202950.0,...,1490.0,-0.032468,1.006236,-0.006443,151.709402,0.108241,OH,2024-06-14,1339.131151,199743.798643
332536,Butler County,14.0,-0.300000,15.0,-0.062500,6.0,0.200000,20.0,0.250000,243400.0,...,2093.0,-0.063953,0.986158,-0.000200,123.274775,-0.013492,KS,2024-08-09,1139.957933,239090.344791
332539,Schuylkill County,40.0,0.333333,34.0,0.096774,12.0,-0.076923,30.0,0.200000,124950.0,...,1504.0,-0.101553,0.984764,-0.010374,79.697371,-0.009550,PA,2024-06-14,1017.609822,138286.398240
332540,Aransas County,14.0,0.400000,14.0,0.555556,0.0,-1.000000,16.0,1.000000,389500.0,...,1402.0,-0.180117,0.961376,0.026082,282.193900,0.310313,TX,2024-06-14,1335.259605,328197.370747


In [11]:
df.describe()

,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,median_new_listing_price,median_new_listing_price_yoy,...,months_of_supply_yoy,median_pending_sqft,median_pending_sqft_yoy,average_sale_to_list_ratio,average_sale_to_list_ratio_yoy,median_sale_ppsf,median_sale_ppsf_yoy,date,ZORI,ZHVI
count,134622.000000,134622.000000,134622.000000,134622.000000,134622.000000,134622.000000,134622.000000,134622.000000,1.346220e+05,134622.000000,...,134622.000000,134622.000000,134622.000000,134622.000000,134622.000000,134622.000000,134622.000000,134622,134622.000000,1.346220e+05
mean,143.551418,0.024969,145.507272,0.010696,39.108786,0.240204,122.145363,0.012234,3.740092e+05,0.080682,...,0.116698,1743.812118,0.010545,0.991758,0.000769,211.186544,0.079820,2021-08-09 16:24:59.237865984,1573.396022,3.523728e+05
min,1.000000,-0.963964,1.000000,-0.995088,0.000000,-1.000000,1.000000,-0.961538,2.657100e+04,-0.930806,...,-460.880347,350.000000,-0.856000,0.600000,-0.394210,16.846484,-0.906676,2016-12-30 00:00:00,566.539368,7.844892e+04
25%,37.000000,-0.171875,39.000000,-0.181818,7.000000,-0.333333,33.000000,-0.196429,2.400000e+05,-0.003322,...,-2.750000,1559.000000,-0.060483,0.977677,-0.009231,133.599658,0.013314,2019-09-06 00:00:00,1203.921355,2.228213e+05
50%,77.000000,-0.016844,80.000000,-0.034884,18.000000,0.000000,68.000000,-0.034483,3.225000e+05,0.067203,...,0.018206,1712.500000,0.002467,0.989796,0.001244,171.377284,0.069473,2022-01-07 00:00:00,1467.637972,2.999117e+05
75%,164.000000,0.150000,168.000000,0.127778,44.000000,0.400000,142.000000,0.146067,4.375000e+05,0.150357,...,2.811109,1888.000000,0.069294,1.003099,0.011511,230.265612,0.136385,2023-08-25 00:00:00,1824.320157,4.152828e+05
max,2608.000000,22.818182,10304.000000,154.000000,1165.000000,57.000000,2679.000000,16.300000,1.075000e+07,8.673529,...,459.794118,7669.000000,3.388926,1.314000,0.436111,3308.823529,8.963946,2024-11-01 00:00:00,16914.169881,2.442129e+06
std,215.277224,0.372939,217.561447,0.680041,64.569779,1.378424,178.483618,0.368326,2.241175e+05,0.165326,...,7.475859,294.942680,0.134873,0.024057,0.021775,146.444823,0.126007,NaN,621.995215,2.053800e+05


In [12]:
# print list of features
df.columns

Index(['region_name', 'adjusted_average_new_listings',
       'adjusted_average_new_listings_yoy',
       'average_pending_sales_listing_updates',
       'average_pending_sales_listing_updates_yoy', 'off_market_in_two_weeks',
       'off_market_in_two_weeks_yoy', 'adjusted_average_homes_sold',
       'adjusted_average_homes_sold_yoy', 'median_new_listing_price',
       'median_new_listing_price_yoy', 'median_sale_price',
       'median_sale_price_yoy', 'median_days_to_close',
       'median_days_to_close_yoy', 'median_new_listing_ppsf',
       'median_new_listing_ppsf_yoy', 'active_listings', 'active_listings_yoy',
       'median_days_on_market', 'median_days_on_market_yoy',
       'percent_active_listings_with_price_drops',
       'percent_active_listings_with_price_drops_yoy', 'age_of_inventory',
       'age_of_inventory_yoy', 'months_of_supply', 'months_of_supply_yoy',
       'median_pending_sqft', 'median_pending_sqft_yoy',
       'average_sale_to_list_ratio', 'average_sale_to_list

In [13]:
# aggregate columns by weighted average
# List of features to be aggregated by weighted mean (based on 'active_listings')

# Example usage with the original DataFrame
final_processed_data = utils.process_data(df.drop(columns='region_name'), utils.features_weighted)

# Display the first few rows of the processed data
print(final_processed_data.head())

        date State  adjusted_average_new_listings  \
0 2016-12-30    AL                      43.000000   
1 2016-12-30    AR                      76.234158   
2 2016-12-30    AZ                    1477.045924   
3 2016-12-30    CA                     612.258489   
4 2016-12-30    CO                     116.748688   

   adjusted_average_new_listings_yoy  average_pending_sales_listing_updates  \
0                          -0.556701                              52.000000   
1                          -0.278601                              66.045595   
2                          -0.054752                            1240.611192   
3                          -0.166035                             494.876903   
4                          -0.142112                             113.713386   

   average_pending_sales_listing_updates_yoy  off_market_in_two_weeks  \
0                                  -0.174603                 3.000000   
1                                  -0.128109                

In [14]:
#Inspect Data
final_processed_data

,date,State,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,...,months_of_supply,months_of_supply_yoy,median_pending_sqft,median_pending_sqft_yoy,average_sale_to_list_ratio,average_sale_to_list_ratio_yoy,median_sale_ppsf,median_sale_ppsf_yoy,ZORI,ZHVI
0,2016-12-30,AL,43.000000,-0.556701,52.000000,-0.174603,3.000000,-0.571429,36.000000,0.161290,...,37.555556,-11.831541,1994.000000,-0.005982,0.962797,-0.005236,99.543540,0.022577,1122.650058,216066.470760
1,2016-12-30,AR,76.234158,-0.278601,66.045595,-0.128109,10.188563,-0.032618,39.914219,-0.178741,...,42.406348,1.690597,1728.596020,0.099004,0.978784,0.030807,91.806029,0.000841,829.503501,148114.863503
2,2016-12-30,AZ,1477.045924,-0.054752,1240.611192,-0.011048,127.216071,-0.097563,689.923260,-0.071023,...,25.212441,0.323869,1672.773113,0.001682,0.978502,0.004452,131.866021,0.098331,1023.148024,226981.871861
3,2016-12-30,CA,612.258489,-0.166035,494.876903,-0.161775,51.993744,-0.202137,449.127889,-0.100053,...,16.392470,1.228898,1565.083335,0.005876,0.987203,0.000618,281.519117,0.052298,1733.103784,424866.748029
4,2016-12-30,CO,116.748688,-0.142112,113.713386,-0.122118,28.049738,0.623202,90.909974,-0.055629,...,15.428519,-1.889838,2021.811483,0.068551,0.989235,-0.001155,162.279162,0.084925,1356.071288,337328.649093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18451,2024-11-01,VT,14.791733,-0.338254,23.347377,0.045872,5.279809,-0.258237,20.976153,0.423200,...,16.609009,-0.323440,1546.756757,-0.162808,0.976849,-0.009048,260.469676,0.141247,1973.753504,406050.835018
18452,2024-11-01,WA,156.067062,0.103779,206.177883,0.143892,65.596130,0.229365,202.261453,0.301700,...,13.174977,-0.520094,1828.096801,0.041181,0.992876,0.003316,376.619631,0.054799,1974.242008,625140.177926
18453,2024-11-01,WI,68.050876,-0.066108,87.356531,0.105742,9.508065,-0.157108,75.407308,0.202432,...,12.331554,-1.519553,1664.514088,-0.025496,0.998287,-0.000775,186.235057,0.083337,1327.831318,321784.470448
18454,2024-11-01,WV,42.000000,-0.363636,69.000000,0.533333,17.000000,0.214286,56.000000,0.600000,...,10.982143,-4.075000,1690.500000,0.043519,0.986842,-0.002456,179.961089,-0.013279,1676.366667,303582.541466


In [15]:
# Save the processed data to a CSV file
final_processed_data.to_csv("../data/final_data.csv", index=False)